### python과 mysql 연동
1. 외부의 라이브러리 설치 (pymysql)
2. 라이브러리 로드 
3. sql 서버의 정보를 등록 
    - 서버의 주소 (localhost)
    - 서버의 포트번호 (3306)
    - sql 유저의 아이디 (root)
    - sql 유저의 패스워드 (1234)
    - 사용할 데이터베이스명 (ubion)
4. sql 서버의 접속
5. sql 쿼리문을 변수에 저장 
6. 쿼리문을 sql 서버에 요청 
7. 쿼리문에 따라서 응답 메시지를 확인
    - select문인 경우에는 응답 메시지를 받아서 데이터프레임으로 변환
8. sql 서버와의 연결을 종료

In [ ]:
import pandas as pd
import pymysql
import dotenv

In [ ]:
import os

In [ ]:
# .env 안에 변수를 환경 변수에 임의로 등록
dotenv.load_dotenv()

In [ ]:
# 환경 변수에 데이터를 로드 
os.getenv('host')

In [ ]:
# 서버의 정보와 접속 
# pymysql 라이브러리 안에 함수를 호출 
# connect(  host = 서버의주소, port = 포트번호, user = 유저id, 
#           password = 유저비밀번호, db = 사용할 데이터베이스명 )
_db = pymysql.connect(
    host = os.getenv('host'), 
    port = int(os.getenv('port')), 
    user = os.getenv('user'), 
    password = os.getenv('pwd'), 
    db = os.getenv('db_name')
)

In [ ]:
# 쿼리문을 작성 
select_sql = """
    SELECT 
    * 
    FROM 
    `sales records`
    limit 5
"""

In [ ]:
# 커서(Cursor) 생성

# 기본값으로 커서 생성 : select문의 응답 데이터의 타입이 2차원 튜플
cursor = _db.cursor()

# DictCursor 사용 : select문의 응답 데이터의 타입이 
# 리스트 안에 딕셔너리 형태 2차원 데이터
cursor2 = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
# 커서에 sql 질의를 보낸다. 
cursor.execute(select_sql)
cursor2.execute(select_sql)

In [ ]:
# 커서에 보낸 질의에 대한 응답을 받아온다. 
# fetchall()
print(cursor.fetchall())

In [ ]:
print(cursor2.fetchall())

In [ ]:
cursor.execute(select_sql)
sql_data = cursor.fetchall()

In [ ]:
pd.DataFrame(sql_data)

In [ ]:
cursor2.execute(select_sql)
sql_data2 = cursor2.fetchall()

In [ ]:
pd.DataFrame(sql_data2)

In [ ]:
insert_sql = """
    INSERT INTO `user` VALUES ("test10", "0011", "choi", 25)
"""

In [ ]:
cursor2.execute(insert_sql)

In [ ]:
select_sql2 = """
    SELECT * FROM `user`
"""
cursor2.execute(select_sql2)
cursor2.fetchall()

In [35]:
_db.commit()

- select함수 생성
    1. 매개변수 1개 (sql 쿼리)
    2. sql 쿼리를 cursor2에 보내는 부분
    3. cursor에서 응답 데이터를 변수에 저장 
    4. 3번과정의 결과를 데이터프레임을 생성 
    5. 데이터프레임을 되돌려준다. 

In [37]:
# 함수를 생성
def select_query(_sql):
    # _sql를 커서에 질의를 보낸다. 
    cursor2.execute(_sql)
    # cursor2에서 응답 데이터를 변수에 저장 
    sql_data = cursor2.fetchall()
    # 데이터프레임로 변경 
    result = pd.DataFrame(sql_data)
    # 결과를 되돌려준다. 
    return result


test_sql = "select * from `user`"
select_query(test_sql)

,id,password,name,age
0,test,1234,kim,20
1,test10,0011,choi,25
2,test2,0000,park,30
3,test3,1111,lee,40
4,test4,12345,kim,35


In [38]:
# 매개변수 2개 함수 : insert, update, delete
# 첫번째 매개변수는 sql 쿼리문
# 두번째 매개변수는 데이터베이스를 수정할것인가? (bool)
def cud_query(_sql, inplace = False):
    # _sql cursor2에 질의를 보낸다. 
    cursor2.execute(_sql)
    # inplace가 True인 경우에는 _db를 동기화
    if inplace:
        _db.commit()
    return "Query OK"

In [39]:
insert_sql2 = """
    insert into `user` values ("test11", "0123", 'kim', 35)
"""
cud_query(insert_sql2)

'Query OK'

In [ ]:
select_query(select_sql2)

In [41]:
_db.commit()

In [42]:
update_sql = """
    update `user` set `password` = "123456789"
    where `id` = "test"
"""
cud_query(update_sql, True)

'Query OK'

In [43]:
# 데이터베이스 서버와의 연결을 종료
_db.close()